## Project: Olympics Data Analysis

### Requirements

- python=3.10 # Specify Python version
- pyspark # PySpark for data processing
- jupyterlab # JupyterLab for interactive development
- ipykernel # Kernel for Jupyter
- pandas # Data manipulation
- numpy # Numerical operations
- matplotlib # Data visualization
- seaborn # Statistical data visualization
- oracledb # Oracle Database client
- sqlalchemy # For Oracle DB connectivity
- pip # Ensure pip is available

### Part 1: Parsing and inserting data

In [11]:
# Importing libraries
import pandas as pd
import cx_Oracle as oracledb

In [12]:
# Loading data
df = pd.read_csv("./olympics_1896_2024.csv", encoding="latin1")

In [13]:
# Checking for specific instances of incorrect encoding and fixing them.
# Replace common misencoded characters with their correct representations.
df.replace({
    r'TÃ¼rkiye': 'Türkiye',
    r'Ã©': 'é',
    r'Ã¶': 'ö',
    r'Ã¼': 'ü',
    r'Ã§': 'ç',
    r'ÃŸ': 'ß',
    r'Ã¡': 'á',
    r'Ã­': 'í',
    r'Ã³': 'ó',
    r'Ãº': 'ú',
    r'Ã±': 'ñ',
    r'Ã€': 'À',
    r'Ãƒ': 'Ã',
    r'Ã•': 'Õ',
    r'â€“': '–',
    r'â€œ': '“',
    r'â€\x9d': '”',
    r'â€\x9c': '“',
    r'â€™': '’',
    r'â€˜': '‘',
}, regex=True, inplace=True)

In [14]:
# Drop rows with missing values
df.dropna(inplace=True)

# Remove rows where 'name' column contains '#NAME?'
df = df[df['name'] != '#NAME?']

In [15]:
# Ensure data types match your database schema
df['discipline'] = df['discipline'].astype(str)
df['event'] = df['event'].astype(str)
df['event_gender'] = df['event_gender'].astype(str)
df['event_type'] = df['event_type'].astype(str)
df['medal_type'] = df['medal_type'].astype(str)
df['name'] = df['name'].astype(str)
df['country_code'] = df['country_code'].astype(str)
df['country'] = df['country'].astype(str)
df['city'] = df['city'].astype(str)
df['event_year'] = df['event_year'].astype(int)  # Ensure this remains numeric

In [16]:
# Oracle DB Connection
connection = oracledb.connect(
    user="system",
    password="hr",
    dsn="127.0.0.1:1521/FREE",
    encoding="UTF-8",
    nencoding="UTF-8"
    # mode=oracledb.SYSDBA
)

In [17]:
cursor = connection.cursor()

cursor.execute("DROP TABLE IF EXISTS OLYMPICS_1896_2024")

# Create Table
cursor.execute("""CREATE TABLE OLYMPICS_1896_2024 (
    discipline VARCHAR2(255),
    event VARCHAR2(200),
    event_gender VARCHAR2(50),
    event_type VARCHAR2(50),
    medal_type VARCHAR2(50),
    name VARCHAR2(200),
    country_code VARCHAR2(10),
    country VARCHAR2(100),
    city VARCHAR2(100),
    event_year NUMBER
)""")

cursor.execute("TRUNCATE TABLE OLYMPICS_1896_2024")

In [18]:
# Insert Data into the Table
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO OLYMPICS_1896_2024 (
            discipline, event, event_gender, event_type, medal_type,
            name, country_code, country, city, event_year
        ) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10)
    """, (
        row['discipline'], row['event'], row['event_gender'], row['event_type'], row['medal_type'],
        row['name'], row['country_code'], row['country'], row['city'], row['event_year']
    ))

# Commit the transaction
connection.commit()

# Close cursor and connection
cursor.close()
connection.close()

print("Data inserted successfully!")

Data inserted successfully!


### Part 2: Retrieving data from the database and creating visualizations

In [19]:
# Importing libraries
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

In [ ]:
# Initializing the pyspark session
spark = SparkSession.builder.appName("OlympicsData")\
    .config("spark.jars", "/absolute/path/to/ojdbc17.jar")\
    .getOrCreate()

In [ ]:
# Creating the connection to the Oracle database and reading data from the OLYMPICS_1896_2024 table
url = "jdbc:oracle:thin:@127.0.0.1:1521/FREE"
properties = {
    "user": "system",
    "password": "hr",
    "driver": "oracle.jdbc.driver.OracleDriver"
}

# Reading data from the OLYMPICS_1896_2024 table
df_db = spark.read.jdbc(url=url, table="OLYMPICS_1896_2024",
                        properties=properties)

df_db = df_db.withColumn("EVENT_YEAR", df_db["EVENT_YEAR"].cast("integer"))
df_db.printSchema()

## Functions to prepare visualization 

In [ ]:
def medal_distribution_by_country():
    """
    Calculates the total medals won by each country using PySpark.
    """
    try:
        result = df_db.groupBy("country") \
            .agg(F.count("medal_type").alias("total_medals")) \
            .orderBy(F.desc("total_medals")) \
            .limit(20)

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "MEDAL_DISTRIBUTION_BY_COUNTRY") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def yearly_medal_trends():
    """
    Calculates the number of medals awarded each year using PySpark.
    """
    try:
        result = df_db.groupBy("event_year") \
            .agg(F.count("medal_type").alias("total_medals")) \
            .orderBy("event_year")

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "YEARLY_MEDAL_TRENDS") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def top_medalists(top_n=10):
    """
    Identifies the top medalists with the most medals using PySpark, 
    ignoring null or NaN values in the name column.
    """
    # Group by "name", aggregate medal counts, and order by total medals
    try:
        result = df_db.groupBy("name") \
            .agg(F.count("medal_type").alias("total_medals")) \
            .orderBy(F.desc("total_medals")) \
            .limit(top_n)

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "TOP_MEDALISTS") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")

    return result.toPandas()

In [ ]:
def medal_distribution_by_gender():
    """
    Calculates the proportion of medals won by gender using PySpark.
    """
    try:
        result = df_db.groupBy("event_gender") \
            .agg(F.count("medal_type").alias("total_medals"))

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "MEDAL_DISTRIBUTION_BY_GENDER") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def medals_by_host_city():
    """
    Calculates the total medals awarded in each host city using PySpark.
    """
    try:
        result = df_db.groupBy("city") \
            .agg(F.count("medal_type").alias("total_medals")) \
            .orderBy(F.desc("total_medals")) \
            .limit(20)

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "MEDALS_BY_HOST_CITY") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def medals_by_country_map():
    """
    Calculates the total medals won by each country for visualization on a map using PySpark.
    """
    try:
        result = df_db.groupBy("country_code") \
            .agg(F.count("medal_type").alias("total_medals"))

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "MEDALS_BY_COUNTRY_MAP") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def medals_by_discipline():
    """
    Calculates the total medals awarded in each discipline using PySpark.
    """
    try:
        result = df_db.groupBy("discipline") \
            .agg(F.count("medal_type").alias("total_medals")) \
            .orderBy(F.desc("total_medals")) \
            .limit(20)

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "MEDALS_BY_DISCIPLINE") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def event_popularity():
    """
    Identifies the most popular events by participation using PySpark.
    """
    try:
        result = df_db.groupBy("event") \
            .agg(F.count("medal_type").alias("total_participants")) \
            .orderBy(F.desc("total_participants")) \
            .limit(20)

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "EVENT_POPULARITY") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def medal_type_proportions():
    """
    Calculates the proportion of each medal type using PySpark.
    """
    try:
        result = df_db.groupBy("medal_type") \
            .agg(F.count("medal_type").alias("total_medals"))

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "MEDAL_TYPE_PROPORTIONS") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def medal_type_trends():
    """
    Tracks changes in medal distribution trends by type over time using PySpark.
    """
    try:
        result = df_db.groupBy("event_year", "medal_type") \
            .agg(F.count("medal_type").alias("total_medals")) \
            .orderBy("event_year", "medal_type")

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "MEDAL_TYPE_TRENDS") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def top_countries_by_athlete_medals():
    """
    Identifies countries with the most unique medalist athletes using PySpark.
    """
    try:
        result = df_db.groupBy("country") \
            .agg(F.countDistinct("name").alias("unique_athletes")) \
            .orderBy(F.desc("unique_athletes")) \
            .limit(20)

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "TOP_COUNTRIES_BY_ATHLETE_MEDALS") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def repeat_medalists():
    """
    Identifies athletes who have won multiple medals using PySpark.
    """
    try:
        result = df_db.groupBy("name") \
            .agg(F.count("medal_type").alias("total_medals")) \
            .filter("total_medals > 1") \
            .orderBy(F.desc("total_medals")) \
            .limit(10)

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "REPEAT_MEDALISTS") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def country_performance_over_time():
    """
    Tracks medal performance of top countries over time using PySpark.
    """
    try:
        top_countries = [row["country"] for row in df_db.groupBy(
            "country").count().orderBy(F.desc("count")).limit(10).collect()]
        result = df_db.filter(F.col("country").isin(top_countries)) \
            .groupBy("event_year", "country") \
            .agg(F.count("medal_type").alias("total_medals")) \
            .orderBy("event_year")

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "COUNTRY_PERFORMANCE_OVER_TIME") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def gender_based_event_trends():
    """
    Tracks changes in the number of events for each gender over time using PySpark.
    """
    try:
        result = df_db.groupBy("event_year", "event_gender") \
            .agg(F.count("event").alias("total_events")) \
            .orderBy("event_year")

        # Write the DataFrame to Oracle database
        result.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "GENDER_BASED_EVENT_TRENDS") \
            .options(**properties) \
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return result.toPandas()

In [ ]:
def host_country_advantage():
    """
    Analyzes medals won by host countries during their hosted games using PySpark.
    """
    try:
        host_medals = df_db.groupBy("city", "country") \
            .agg(F.count("medal_type").alias("total_medals"))

        # Write the DataFrame to Oracle database
        host_medals.write \
            .format("jdbc") \
            .option("url", url) \
            .option("dbtable", "HOST_COUNTRY_ADVANTAGE") \
            .options(**properties)\
            .mode("overwrite") \
            .save()

    except Exception as e:
        print(f"An error occurred: {e}")
    return host_medals.toPandas()